In [1]:
## Import warnings. Supress warnings (for  matplotlib)
import warnings
warnings.simplefilter("ignore")

In [73]:
## Import analysis modules
import pandas as p
from pandas.tools.plotting import scatter_matrix
from numpy import nan, isnan, mean, std, hstack, ravel
from sklearn.cross_validation import train_test_split, cross_val_score, KFold, LeaveOneOut, LeavePOut, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, Binarizer, Imputer, \
LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, roc_curve, auc
from sklearn.grid_search import GridSearchCV

## Import visualization modules
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

## Import SciPy
from scipy.sparse import issparse

In [3]:
## Read in file
data = p.read_csv('F:\\DePaul\\IS467\\Week5\\loan.csv',delimiter='~}',na_values='nan',)

In [4]:
## Count of instances and features
rows, columns = data.shape
print data.shape

(42535, 112)


In [5]:
## Get basic statistics for continuous features
numeric = data.describe(include=['number']).T.reset_index()
numeric.rename(columns={'index':'feature'},inplace=True)
numeric.insert(1,'missing',(rows - numeric['count'])/ float(rows))

In [6]:
## How many features can we eliminate?
drop = numeric[(numeric['missing']==1) | (numeric['std']==0)]

In [7]:
## Drop the unhelpful features from the base and numeric table
data = data.drop(drop['feature'],axis=1)
numeric = numeric[~numeric['feature'].isin(drop['feature'])]

In [8]:
## Get basic statistics for discrete features
discrete = data.describe(include=['object']).T.reset_index()
discrete.rename(columns={'index':'feature'},inplace=True)
discrete.insert(1,'missing',(rows - discrete['count'])/ float(rows))

In [9]:
## How many features can we eliminate?
ddrop = discrete[(discrete['missing']>.6) | (discrete['unique']==1)]

In [10]:
## Drop unhelpful features from the base table
data = data.drop(ddrop['feature'],axis=1)
discrete = discrete[~discrete['feature'].isin(ddrop['feature'])]

In [11]:
## How many columns do we have left?
data.shape

(42535, 51)

In [12]:
## Double check discrete
discrete

,feature,missing,count,unique,top,freq
0,term,0,42535,2,36 months,31534
1,int_rate,0,42535,394,10.99%,970
2,grade,0,42535,7,B,12389
3,sub_grade,0,42535,35,B3,2997
4,emp_title,0.0616904,39911,30659,US Army,139
5,emp_length,0,42535,12,10+ years,9369
6,home_ownership,0,42535,5,RENT,20181
7,verification_status,0,42535,3,Not Verified,18758
8,issue_d,0,42535,55,Dec-2011,2267
9,loan_status,0,42535,9,Fully Paid,33314


In [13]:
## Discrete remove
data = data.drop(['grade','sub_grade','int_rate','emp_title','issue_d','pymnt_plan','url','desc','title','earliest_cr_line','last_pymnt_d','last_credit_pull_d'],axis=1)

In [14]:
## Check numeric
numeric

,feature,missing,count,mean,std,min,25%,50%,75%,max
0,id,0.000000,42535.0,664579.852310,219302.219319,54734.00,498392.500000,644250.000000,8.258225e+05,1.077501e+06
1,member_id,0.000000,42535.0,825702.551170,279540.905635,70473.00,638479.500000,824178.000000,1.033946e+06,1.314167e+06
2,loan_amnt,0.000000,42535.0,11089.722581,7410.938391,500.00,5200.000000,9700.000000,1.500000e+04,3.500000e+04
3,funded_amnt,0.000000,42535.0,10821.585753,7146.914675,500.00,5000.000000,9600.000000,1.500000e+04,3.500000e+04
4,funded_amnt_inv,0.000000,42535.0,10139.830603,7131.686446,0.00,4950.000000,8500.000000,1.400000e+04,3.500000e+04
5,installment,0.000000,42535.0,322.623063,208.927216,15.67,165.520000,277.690000,4.281800e+02,1.305190e+03
6,annual_inc,0.000094,42531.0,69136.556420,64096.349719,1896.00,NaN,NaN,NaN,6.000000e+06
7,dti,0.000000,42535.0,13.373043,6.726315,0.00,8.200000,13.470000,1.868000e+01,2.999000e+01
8,delinq_2yrs,0.000682,42506.0,0.152449,0.512406,0.00,NaN,NaN,NaN,1.300000e+01
9,inq_last_6mths,0.000682,42506.0,1.081424,1.527455,0.00,NaN,NaN,NaN,3.300000e+01


In [15]:
data = data.drop(['id','member_id'],axis=1)

In [16]:
data.shape

(42535, 37)

In [17]:
## Keep only those loan statuses where fully paid or charged off
data = data[data['loan_status'].isin(['Fully Paid','Charged Off'])]

In [18]:
sparse_cols = ['delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec',
          'total_acc','out_prncp','out_prncp_inv','total_rec_late_fee','recoveries','collection_recovery_fee', 
          'acc_now_delinq','delinq_amnt','pub_rec_bankruptcies', 'tax_liens']

In [19]:
discrete_cols = ['term','emp_length','home_ownership','verification_status', 'purpose', 
            'zip_code', 'addr_state']


In [20]:
numeric_cols = [x for x in data.columns if x not in sparse_cols + discrete_cols + ['loan_status']]

In [21]:
## Address by stripping leading space
data['term'] = data['term'].str.strip()

In [22]:
## Scikit learn estimators require numeric features
term_map = {'36 months':0,'60 months':1}
emp_map = {'n/a':0,'< 1 year':1,'1 year':2,'2 years':3,'3 years':4,'4 years':5,'5 years':6,'6 years':7,'7 years':8,'8 years':9,
           '9 years':10, '10 years':11}
status_map = {'Fully Paid':0,'Charged Off':1}

In [23]:
## Convert categorical features to numeric using mapping function
data['term'] = data['term'].map(term_map)
data['emp_length'] = data['emp_length'].map(emp_map)
data['loan_status'] = data['loan_status'].map(status_map)

In [24]:
data['emp_length'].fillna(0.0, inplace=True)

In [25]:
## Leverage regular expressions to clean revol_util and int_rate
data['revol_util'].replace('%','',regex=True,inplace=True)

In [26]:
## Convert revol_util to numeric 
data['revol_util'] = p.to_numeric(data['revol_util'])

In [27]:
data['revol_util'].fillna(0.0, inplace=True)

In [28]:
validation = data.sample(frac=.2,random_state=12345)
val_x = validation.drop('loan_status',axis=1)
val_y = validation['loan_status'].as_matrix()

In [29]:
new_data = data.drop(validation.index,axis=0)

In [30]:
## Seperate input features from target feature
x = new_data.drop('loan_status',axis=1)
y = new_data['loan_status'].as_matrix()

In [31]:
## Take a look at x
x.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,...,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000,5000,4975.0,0,162.87,0.0,RENT,24000.0,Verified,credit_card,...,5000.0,863.16,0.00,0.0,0.0,171.62,0.0,0.0,0.0,0.0
2,2400,2400,2400.0,0,84.33,0.0,RENT,12252.0,Not Verified,small_business,...,2400.0,605.67,0.00,0.0,0.0,649.91,0.0,0.0,0.0,0.0
3,10000,10000,10000.0,0,339.31,0.0,RENT,49200.0,Source Verified,other,...,10000.0,2214.92,16.97,0.0,0.0,357.48,0.0,0.0,0.0,0.0
5,5000,5000,5000.0,0,156.46,4.0,RENT,36000.0,Source Verified,wedding,...,5000.0,632.21,0.00,0.0,0.0,161.03,0.0,0.0,0.0,0.0
7,3000,3000,3000.0,0,109.43,10.0,RENT,48000.0,Source Verified,car,...,3000.0,939.14,0.00,0.0,0.0,111.34,0.0,0.0,0.0,0.0


In [32]:
## Take a look at y
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [33]:
## Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.3,random_state=15254)

In [34]:
## Impute missing cases using scikit learn
imp = Imputer(missing_values='NaN',strategy='most_frequent',axis=0)
for col in sparse_cols:
    imp.fit(x_train[col].reshape(-1,1))
    x_train[col] = imp.transform(x_train[col].reshape(-1,1))
    x_test[col] = imp.transform(x_test[col].reshape(-1,1))
    val_x[col] = imp.transform(val_x[col].reshape(-1,1))

In [35]:
mas = MaxAbsScaler()
for col in sparse_cols:
    mas.fit(x_train[col].reshape(-1,1))
    x_train[col] = mas.transform(x_train[col].reshape(-1,1))
    x_test[col] = mas.transform(x_test[col].reshape(-1,1))
    val_x[col] = mas.transform(val_x[col].reshape(-1,1))

In [36]:
cols = ['home_ownership','verification_status','purpose','zip_code','addr_state']
le = LabelEncoder()
for col in cols:
    le.fit(ravel(data[col]))
    data[col] = le.transform(ravel(data[col]))
    x_train[col] = le.transform(ravel(x_train[col]))
    x_test[col] = le.transform(ravel(x_test[col]))
    val_x[col] = le.transform(ravel(val_x[col]))

In [37]:
## Standard histograms with pandas
rb=RobustScaler()
st=StandardScaler()
for col in numeric_cols:
    if col in ['annual_inc','revol_bal']:
        rb.fit(x_train[col].reshape(-1,1))
        x_train[col] = rb.transform(x_train[col].reshape(-1,1))
        x_test[col] = rb.transform(x_test[col].reshape(-1,1))
        val_x[col] = rb.transform(val_x[col].reshape(-1,1))
    else:
        st.fit(x_train[col].reshape(-1,1))
        x_train[col] = st.transform(x_train[col].reshape(-1,1))
        x_test[col] = st.transform(x_test[col].reshape(-1,1))
        val_x[col] = st.transform(val_x[col].reshape(-1,1))

C:\Users\avelkoski\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avelkoski\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avelkoski\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avelkoski\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\avelkoski\AppData\L

In [38]:
x_train.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,...,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,acc_now_delinq,delinq_amnt,pub_rec_bankruptcies,tax_liens
39604,0.526160,0.582997,-1.403175,0,0.778431,2.0,4,-0.209524,0,2,...,0.744848,0.132981,0.0,0.0,0.0,-0.498204,0.0,0.0,0.0,0.0
35537,-0.149733,-0.117829,-0.039121,0,0.103150,1.0,4,-0.685714,1,2,...,0.037794,-0.734830,0.0,0.0,0.0,-0.604938,0.0,0.0,0.0,0.0
6361,-1.095982,-1.098986,-1.028590,0,-1.053142,0.0,4,-0.066667,1,7,...,-0.952082,-0.596584,0.0,0.0,0.0,-0.536952,0.0,0.0,0.0,0.0
5124,1.202052,1.283822,1.374406,1,0.712208,6.0,0,-0.090476,1,4,...,1.451903,0.721944,0.0,0.0,0.0,2.904305,0.0,0.0,0.0,0.0
5027,-0.149733,-0.117829,-0.039121,0,-0.027000,7.0,0,0.028571,0,4,...,0.037794,-0.299891,0.0,0.0,0.0,-0.534943,0.0,0.0,0.0,0.0


In [39]:
ohe = OneHotEncoder()
ohe.fit(data.loc[:,discrete_cols])
x_train_discrete = ohe.transform(x_train.loc[:,discrete_cols]).toarray()
x_test_discrete = ohe.transform(x_test.loc[:,discrete_cols]).toarray()
val_discrete = ohe.transform(val_x.loc[:,discrete_cols]).toarray()

In [40]:
## Lets's try to extract components via PCA 
pca = PCA(n_components=5)
pca.fit(x_train.loc[:,numeric_cols])

PCA(copy=True, n_components=5, whiten=False)

In [41]:
## Percentage of variance explained by each of the selected components.
print(['%0.2f' % z for z in pca.explained_variance_ratio_]) 

['0.52', '0.15', '0.11', '0.06', '0.05']


In [42]:
## Transform x
x_train_pca = pca.transform(x_train.loc[:,numeric_cols])
x_test_pca = pca.transform(x_test.loc[:,numeric_cols])
val_x_pca = pca.transform(val_x.loc[:,numeric_cols])

In [43]:
x_train_sparse = x_train.loc[:,sparse_cols].as_matrix()
x_test_sparse = x_test.loc[:,sparse_cols].as_matrix()
val_x_sparse = val_x.loc[:,sparse_cols].as_matrix()

In [44]:
x_train_final = hstack([x_train_pca,x_train_discrete,x_train_sparse])
x_test_final = hstack([x_test_pca,x_test_discrete,x_test_sparse])
val_x_final = hstack([val_x_pca,val_discrete,val_x_sparse])

In [45]:
print x_train_final.shape
print x_test_final.shape
print val_x_final.shape

(21814L, 928L)
(9349L, 928L)
(7791L, 928L)


In [46]:
## Create estimator
clf = DecisionTreeClassifier(class_weight='balanced',max_depth=3)

In [64]:
## Fit the model using training set 
clf.fit(x_train_final,y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [65]:
## Check accuracy score
print '%0.2f' % clf.score(x_test_final,y_test)

0.96


In [66]:
## Predict y given test set
predictions = clf.predict(x_test_final)

In [67]:
## Take a look at the confusion matrix ([TN,FN],[FP,TP])
confusion_matrix(y_test,predictions)

array([[7922,   65],
       [ 331, 1031]])

In [68]:
## Accuracy score
print '%0.2f' % precision_score(y_test, predictions)

0.94


In [69]:
## Recall score
print '%0.2f' % recall_score(y_test, predictions)

0.76


In [70]:
## Print classification report
print classification_report(y_test, predictions)

             precision    recall  f1-score   support

          0       0.96      0.99      0.98      7987
          1       0.94      0.76      0.84      1362

avg / total       0.96      0.96      0.96      9349



In [71]:
## Get data to plot ROC Curve
fp, tp, th = roc_curve(y_test, predictions)
roc_auc = auc(fp, tp)

In [72]:
## Plot ROC Curve
plt.title('ROC Curve')
plt.plot(fp, tp, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [56]:
## Check accuracy score
print '%0.2f' % clf.score(val_x_final,val_y)

0.95


In [57]:
## Predict y given test set
predictions = clf.predict(val_x_final)

In [58]:
## Take a look at the confusion matrix ([TN,FN],[FP,TP])
confusion_matrix(val_y,predictions)

array([[6594,   68],
       [ 284,  845]])

In [59]:
## Accuracy score
print '%0.2f' % precision_score(val_y, predictions)

0.93


In [60]:
## Recall score
print '%0.2f' % recall_score(val_y, predictions)

0.75


In [61]:
## Print classification report
print classification_report(val_y, predictions)

             precision    recall  f1-score   support

          0       0.96      0.99      0.97      6662
          1       0.93      0.75      0.83      1129

avg / total       0.95      0.95      0.95      7791



In [62]:
## Get data to plot ROC Curve
fp, tp, th = roc_curve(val_y, predictions)
roc_auc = auc(fp, tp)

In [63]:
## Plot ROC Curve
plt.title('ROC Curve')
plt.plot(fp, tp, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [75]:
params = {'class_weight':['balanced',None],
          'criterion':['gini','entropy'],
          'max_depth':[2,3,4],
          'max_features':['auto','sqrt','log2',None],
          'min_samples_leaf':[1,2,3],
          'min_samples_split':[2,4]
}

In [81]:
## Create estimator
clf = DecisionTreeClassifier()

In [99]:
gs = GridSearchCV(clf,param_grid=params,scoring='recall')

In [100]:
gs.fit(x_train_final,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_leaf': [1, 2, 3, 4, 5], 'min_samples_split': [2, 4, 6], 'criterion': ['gini', 'entropy'], 'max_features': ['auto', 'sqrt', 'log2', None], 'max_depth': [2, 3, 4, 5], 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [87]:
gs.best_score_

0.98603025010544065

In [88]:
gs.best_estimator_

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=2,
            max_features='log2', max_leaf_nodes=None, min_samples_leaf=4,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [89]:
gs.scorer_

make_scorer(recall_score)

In [90]:
est = gs.best_estimator_

In [91]:
## Fit the model using training set 
est.fit(x_train_final,y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=2,
            max_features='log2', max_leaf_nodes=None, min_samples_leaf=4,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [93]:
## Check accuracy score
print '%0.2f' % est.score(x_test_final,y_test)

0.35


In [94]:
## Predict y given test set
predictions = est.predict(x_test_final)

In [95]:
## Take a look at the confusion matrix ([TN,FN],[FP,TP])
confusion_matrix(y_test,predictions)

array([[1928, 6059],
       [  61, 1301]])

In [96]:
## Accuracy score
print '%0.2f' % precision_score(y_test, predictions)

0.18


In [97]:
## Recall score
print '%0.2f' % recall_score(y_test, predictions)

0.96


In [98]:
## Print classification report
print classification_report(y_test, predictions)

             precision    recall  f1-score   support

          0       0.97      0.24      0.39      7987
          1       0.18      0.96      0.30      1362

avg / total       0.85      0.35      0.37      9349



In [71]:
## Get data to plot ROC Curve
fp, tp, th = roc_curve(y_test, predictions)
roc_auc = auc(fp, tp)

In [72]:
## Plot ROC Curve
plt.title('ROC Curve')
plt.plot(fp, tp, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()